In [152]:
import pandas as pd
from bs4 import BeautifulSoup
import re

In [170]:
counties = set(pd.read_csv('uk-counties-list.csv', header=None)[1]) #.str.lower())
extended_counties = counties.union({'Middlesex', 'Southampton'})

In [3]:
with open('tier2.xml', encoding='utf-8') as f:
    file = f.read()

In [5]:
soup = BeautifulSoup(file, parser='xml')

In [25]:
start_tag = soup.find('text', string='County')

In [221]:
column_names = {'County', 'Tier & Rating', 'Organisation Name', 'Sub Tier', 'Town/City'}
def next_info_tag(tag):
    tag = tag.find_next('text')
    if tag is None: return tag
    while tag.text in column_names or re.match('^Page \d* of \d*$', tag.text):
        tag = tag.find_next('text')
    if tag and tag.text == 'Total' and tag.find_next('text').text == 'Tier 2':
        tag = None
    return tag

sponsors = []
tag = next_info_tag(start_tag)
tier_subtypes = {'Creative & Sporting', 'Tier 2 General', 'Seasonal Worker', 
                 'Intra Company Transfers (ICT)', 'Religious Workers', 'Voluntary Workers',
                 'Exchange', 'Sport', 'International Agreements'}

curr_row = {}
not_in_list_county = []
while tag:
# for i in range(100):
    if tag.find('b'):
        curr_row['name'] = tag.text
    else:
        print(f'SOMETHING WRONG: {tag} is not  organization name')
        continue
    tag = next_info_tag(tag)
    if tag.text.strip(',') in extended_counties:
        curr_row['county'] = tag.text
        tag = next_info_tag(tag)
    next_tag = next_info_tag(tag)
    if (next_tag.text not in tier_subtypes) and (not next_tag.text.startswith('Tier ')):
        curr_row['county'] = tag.text
        not_in_list_county.append(tag.text)
        tag = next_info_tag(tag)
       
    curr_row['city'] = tag.text
    tag = next_info_tag(tag)
        
    tier_options = [{}] # list of visa types dict
    while tag and not tag.find('b'):
        type_rate = re.search('(Tier \d).*?\((.*)\)', tag.text)
        if type_rate:
            tier_options[-1]['tier_type'] = type_rate.groups()[0]
            tier_options[-1]['tier_rating'] = type_rate.groups()[1]
            tag = next_info_tag(tag)
            if (not 'tier_subtype' in tier_options[-1]) and (not tag.find('b')):
                continue
        elif tag.text in tier_subtypes:
            tier_options[-1]['tier_subtype'] = tag.text
            tag = next_info_tag(tag)
            if (not 'tier_type' in tier_options[-1]) and (not tag.find('b')):
                continue
        else:
            print(tag.text)
            tag = next_info_tag(tag)
        tier_options.append({})
    for tier_option in tier_options[:-1]:
        if 'tier_rating' not in tier_option:
            print(curr_row)
        sponsors.append({**curr_row, **tier_option})

    curr_row = {}

 
{'name': 'Manchester Trinity College Limited', 'city': 'Manchester'}


In [224]:
# sponsors_df = sponsors_df.append({**curr_row, **tier_option}, ignore_index=True)
columns = ['name', 'city', 'county', 'tier_type', 'tier_rating', 'tier_subtype']
sponsors_df = pd.DataFrame(sponsors, columns=columns)

In [230]:
s = sponsors_df['tier_rating'].unique()
sponsors_df[sponsors_df.tier_type == 'Tier 5'].nunique()

name            3949
city            1091
county           314
tier_type          1
tier_rating        4
tier_subtype       8
dtype: int64

In [219]:
sponsors_df.tail(20)

,name,city,county,tier_type,tier_rating,tier_subtype
41050,Zuken Group Limited,Bristol,NaN,Tier 2,A rating,Intra Company Transfers (ICT)
41051,Zuken Group Limited,Bristol,NaN,Tier 2,A rating,Tier 2 General
41052,ZUKER PROPERTY LTD,LONDON,NaN,Tier 2,A rating,Tier 2 General
41053,Zuma Array Limited,London,NaN,Tier 2,A rating,Tier 2 General
41054,Zuora UK Limited,London,NaN,Tier 2,A rating,Tier 2 General
41055,Zuora UK Limited,London,NaN,Tier 2,A rating,Intra Company Transfers (ICT)
41056,Zuri Technologies Ltd,London,NaN,Tier 2,A rating,Tier 2 General
41057,Zurich Insurance Group,Swindon,Wiltshire,Tier 2,A rating,Tier 2 General
41058,Zurich Insurance Group,Swindon,Wiltshire,Tier 2,A rating,Intra Company Transfers (ICT)
41059,Zwift UK Limited,London,NaN,Tier 2,A rating,Tier 2 General
